# Lab 1: Multi-Metallicity Populations, Transient Populations, and Initial Conditions Reweighting
#### Mike Zevin, Monica Gallegos-Garcia

Goals:

* Learn how to combine runs at different metallicities
* Explore how predicted source properties vary across metallicity
* Create transient populations using POSYDON
* Learn how to reweight populations with different initial conditions assumptions

___

In this lab, we are going to begin our exploration of multi-metallicity populations. Metallicity has a major impact on the evolution of massive stars, the transients they generate, and the compact objects they leave behind. Non-local astrophysical sources and transients generally occur at a range of metallicities; for transients that we observe over a large range of redshift, we are observing the universe at a wide range of times and metal content. Thus, when investigating cosmological rates and population properties of astrophysical events (e.g., gravitational-wave sources, gamma-ray bursts, supernovae), taking into account the metallicity evolution of the universe is imperative. 

For the first lab, we will be exploring how POSYDON compiles and handles populations at various metallicities. We will be using POSYDON populations of 10,000 binaries at 8 discrete metallicities. We will also learn about a nifty tool in POSYDON that allows us reweight populations based on different initial condition assumptions, which means you can explore different initial condition assumptions without rerunning entire populations!
___

## 1. Compiling populations run at different metallicities

We will start by reading in the populations that were run at 8 discrete metallicities. For completeness, below is the script that copied over and edited the default initialization file, and ran the populations. These populations took about 10 hours total on a single processor --- not too bad but probably not something you want to do right now! Following this cell, we can just read in the pre-computed populations. 

In [ ]:
import os
import shutil
from posydon.config import PATH_TO_POSYDON
from posydon.popsyn.synthetic_population import PopulationRunner

# copy over default ini file to working directory
path_to_params = os.path.join(PATH_TO_POSYDON, "posydon/popsyn/population_params_default.ini")
shutil.copyfile(path_to_params, './population_params.ini')

# edit the ini file to specify the metallicities we'll run at and increase the number of binaries
!sed -i '' "s/metallicity = \[1.\]/metallicity = [2., 1., 0.45, 0.2, 0.1, 0.01, 0.001, 0.0001]/g" ./population_params.ini
!sed -i '' "s/number_of_binaries = 10/number_of_binaries = 10000/g" ./population_params.ini

# instantiate the PopulationRunner object
poprun = PopulationRunner('./population_params.ini', verbose=True)

# run the population
# poprun.evolve()   # --- commented out for now!

### 1.1 - Prepping the data

Let's go ahead and read in the pre-evolved populations! Let's start by looking for black hole mergers within these populations. We'll take advantage of the `export_selection` method within the population class to create a new file where we'll append the information of all the binary black hole mergers in all the metallicity runs. 

<div class="alert alert-success">

## MINI-EXERCISE:

1. Write the function `merging_BH_indices` below that will extract the indices of merging BBHs from the population histories in the loop below
   
</div>

<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Hint (click to reveal):</summary></b>
    
You might want to filter the `pop.history` on `'S1_state'`, `'S2_state'`, and `'event'`.
    
</details>

In [ ]:
import numpy as np
import pandas as pd
from posydon.popsyn.synthetic_population import Population
from posydon.config import PATH_TO_POSYDON_DATA

pop_files = ['1e-04_Zsun_population.h5',
             '1e-03_Zsun_population.h5',
             '1e-02_Zsun_population.h5',
             '1e-01_Zsun_population.h5',
             '2e-01_Zsun_population.h5',
             '4.5e-01_Zsun_population.h5',
             '1e+00_Zsun_population.h5',
             '2e+00_Zsun_population.h5']

def merging_BH_indices(pop):
    """
    Function to find the indices of systems that merge as black holes within the population

    Returns list of indices in the population `pop` where the binary system led to a BBH merger
    """
    return list(selected_indices)


# importing path to our lab data
data_path = os.path.join(os.path.dirname(PATH_TO_POSYDON_DATA), "2025_school_data/populations/10K_pops")

for file in pop_files:
    file_path = os.path.join(data_path, file)
    pop = Population(file_path)

    # function that gets the indices of systems that merge as BBHs
    selected_indices = merging_BH_indices(pop)

    # print number of systems at this metallicity that merge as BBHs
    print(f'File: {file}, Number of systems: {len(selected_indices)}')

    # let's also determine the formation channels for each system so we have them for later
    pop.calculate_formation_channels(mt_history=True)

    # export these systems to a new file called 'BBH_mergers.h5'
    pop.export_selection(selected_indices, 'BBH_mergers.h5', append=True)

<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Solution (click to reveal):</summary></b>

```python
def merging_BH_indices(pop):
    """
    Function to find the indices of systems that merge as black holes within the population

    Returns list of indices in the population `pop` where the binary system led to a BBH merger
    """
    selected_indices = pop.history.select(where='S1_state == BH & S2_state == BH & event == CO_contact').index
    return list(selected_indices)

```
    
</details>

##### Note: Alternatively, we could have masked our population like you did in Tuesday's lab.
##### Note: it's ok that it says `Missing ini parameter: orbital_separation...` since we defined an orbital period scheme

### 1.2 - Metadata of your multi-metallicity population

We now have a merged population of BBH mergers across our 8 metallicity runs. Let's explore some of the features and metadata that are stored along with this population. 

First, we can read in the merged population file that we just saved. 

In [ ]:
BBH_pop = Population('BBH_mergers.h5')

If you run `BBH_pop.oneline` or `BBH_pop.history`, we can see the oneline and history for all the BBH mergers in our 8 metallicity populations. Pretty convenient! 

The merged population also comes with a number of handy attributes. For example, we can look at the simulated mass and number of systems in each metallicity using the `mass_per_metallicity` attribute. 

##### NOTE: It is often helpful to look at all of the methods/attributes of a Python object. You can do this, for example, by `BBH_pop.__dict__.keys()`

In [ ]:
BBH_pop.mass_per_metallicity

We can also look at the parameters that defined our population sampling with `BBH_pop.ini_params`. This will be useful a little later in the lab when we explore initial conditions reweighting. 

In [ ]:
BBH_pop.ini_params

### 1.3 - Metallicity dependence

Metallicity has a major impact on the resultant remnant that is formed from a massive star, the remnant mass, binary stellar evolution processes, and the efficiency of forming compact binary systems. Here, we'll take a quick look at the objects in our populations that result from different metallicity populations. 

Let's first plot the primary and secondary masses for each BBH system in our multi-metallicity population. 

In [ ]:
import matplotlib.pyplot as plt

# get line of history that has BBH information
history_temp = BBH_pop.history.select(where='S1_state == BH & S2_state == BH & event == CO_contact')

# loop over metallicities
fig, axs = plt.subplots(figsize=(5,5))
for idx, met in enumerate(BBH_pop.solar_metallicities):
    met_mask = np.isclose(BBH_pop.oneline['metallicity'], met)
    BBH_properties = history_temp[met_mask]
    axs.scatter(BBH_properties['S1_mass'], BBH_properties['S2_mass'], s=1, label=r'$Z = %0.4f Z_\odot$' % met)

# plot 1-1 mass line
axs.plot(np.linspace(0,70,2), np.linspace(0,70,2), color='k', alpha=0.2)

# format
axs.set_xlabel(r'$m_1\ [M_\odot]$')
axs.set_ylabel(r'$m_2\ [M_\odot]$')
axs.set_xlim(0,65)
axs.set_ylim(0,65)
plt.legend(frameon=True)

You'll notice that lower metallicities are much more efficient at yielding high-mass (>30 Msun) black holes, whereas at higher metallicities (>0.2 $Z_\odot$) most of the black holes are much lower mass (<20 $M_\odot$). Another thing you might notice (or noticed before when running `BBH_pop.mass_per_metallicity`) is that higher metallicities are much less efficient at forming black holes. 

Let's also take a look at the formation efficiency of merger BBHs as a function of metallicity. 

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(10,4))

# loop over metallicities
for idx, met in enumerate(BBH_pop.solar_metallicities):
    Nsys = BBH_pop.mass_per_metallicity.loc[met,'number_of_systems']
    Msim = BBH_pop.mass_per_metallicity.loc[met,'simulated_mass']
    met_indices = np.where(BBH_pop.oneline['metallicity']==met)[0]
    BBH_oneline = BBH_pop.oneline.select(met_indices)
    Mbbh_progenitors = np.sum(BBH_oneline['S1_mass_i']) + np.sum(BBH_oneline['S2_mass_i'])

    # plot two types of formation efficiency
    axs[0].scatter(met, Nsys/Msim, color='m')
    axs[1].scatter(met, Mbbh_progenitors/Msim, color='c')

# format
for ax in axs:
    ax.set_xlabel(r'$Z/Z_\odot$')
    ax.set_xscale('log')
    ax.set_ylim(bottom=0)
axs[0].set_ylabel(r'Formation Efficiency ($N / M_\odot^\mathrm{samp}$) [$M_\odot^{-1}$]')
axs[1].set_ylabel(r'Formation Efficiency ($M / M_\odot^\mathrm{samp}$) ')

plt.tight_layout()

You can certainly tell that lower metallicities are much more efficient at forming BBH systems. However, these figures are a bit deceiving! This only accounts for the _sampled_ mass, and not the total underlying mass. That is, it doesn't count for the regions of the initial mass function that are not simulated (like low-mass stars <7 $M_\odot$, see `BBH_pop.ini_params`). To get the total underlying mass that was sampled from the full stellar population to get this sample, we'll need to take an extra step. 

<div class="alert alert-success">

## MINI-EXERCISE:

1. Another thing we can look at is how the relative fraction of formation channels vary with metallicity. Let's consider the main two channels for BBH formation: stable mass transfer (where the binary first proceeds through two phases of stable mass transfer) and common envelope (where the binary first proceeds through a stable mass transfer phase, then a common envelope phase between the secondary star and the black hole). Plot the relative fraction of systems that proceed through each of these as a function of metallicity. 
   
</div>

<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Hint (click to reveal):</summary></b>

You can look at the number of systems that proceed through each unique formation channel using `BBH_pop.formation_channels.channel.value_counts()`. Systems that also go through a contact phase in RLO1 can also be considered part of the stable mass transfer category.
    
</details>

<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Solution (click to reveal):</summary></b>

```python
fig, axs = plt.subplots(figsize=(6,6))

# loop over metallicities
for idx, met in enumerate(BBH_pop.solar_metallicities):
    met_indices = np.where(BBH_pop.oneline['metallicity']==met)[0]

    Ntot = len(met_indices)
    all_channels = BBH_pop.formation_channels.channel[met_indices].values
    N_CE = np.sum(all_channels == 'ZAMS_CC1_oRLO2_oCE2_CC2_END')
    N_SMT = np.sum(all_channels == 'ZAMS_oRLO1_CC1_oRLO2_CC2_END') + np.sum(all_channels == 'ZAMS_oRLO1-contact_CC1_oRLO2_CC2_END')

    # plot two types of formation efficiency
    lbl0 = 'CE' if idx==0 else None
    lbl1 = 'SMT' if idx==0 else None
    axs.scatter(met, N_CE/Ntot, color='m', marker='x', label=lbl0)
    axs.scatter(met, N_SMT/Ntot, color='c', marker='^', label=lbl1)

# format
axs.set_xscale('log')
axs.set_ylim(0,1)
axs.set_xlabel(r'$Z/Z_\odot$')
axs.set_ylabel(r'Fraction of Merging BBH Systems')

plt.tight_layout()
```
    
</details>

___
## 2. Transient Populations

We'll first work with a new object, the `TransientPopulation` class. This class allows you to write your own 'creation function' to parse out a transient population of your choosing, and save information that you're interested in retaining for this population. 

### 2.1 - Creating a transient population

We'll write a function to select BBH mergers and save information that is relevant to gravitational-wave data analysis, such as the chirp mass and effective spin. There are some internal functions to calculate these, but you could also write your own functions. We'll also include other relevant information, like metallicity, delay time (time between stellar formation and merger), and inspiral time (time between CBC formation and merger). The cell below is a function that calculates properties of BBHs that merge in a Hubble time from the (multi-metallicity) history and oneline. You can do this in chunks to make memory usage a little easier, but for our small 10k/metallicity population we can just feed it the full dataframe. 

<div class="alert alert-success">

## MINI-EXERCISE:

1. Below is part of a 'creation function' for creating a transient population of BBH mergers. We'll want to include other pertinent information about the BBH merger population, all of which you can get from the history and oneline files:
    - S1/S2 state
    - S1/S2 mass
    - S1/S2 spin
    - S1/S2 spin tilt (`spin_orbit_tilt_merger`)
    - chirp mass (there is a POSYDON function for this if you don't want to write it out, which is imported below)
    - mass ratio (there is a POSYDON function for this if you don't want to write it out, which is imported below)
    - effective spin (there is a POSYDON function for this if you don't want to write it out, which is imported below)
   
</div>

In [ ]:
from posydon.popsyn.synthetic_population import TransientPopulation
from posydon.popsyn.transient_select_funcs import chi_eff, mass_ratio, m_chirp

def BBH_creation_function(history_chunk, oneline_chunk, formation_channels_chunk):
    '''A BBH creation function to create a transient population of BBHs mergers.'''

    indices = oneline_chunk.index.to_numpy()
    df_transients = pd.DataFrame(index = indices)

    df_transients['time'] = history_chunk[history_chunk['event'] == 'CO_contact']['time'] * 1e-6 #Myr
    mask = (history_chunk['S1_state'] == 'BH') & (history_chunk['S2_state'] == 'BH') & (history_chunk['step_names'] == 'step_SN') & (history_chunk['state'] == 'detached')
    df_transients['metallicity'] = oneline_chunk['metallicity']
    df_transients['t_inspiral'] = df_transients['time'] - history_chunk[mask]['time']*1e-6

    # FILL IN RELEVANT INFORMATION HERE, SEE BELOW TO COMPARE WHAT YOU HAVE!

    # include formation channel
    df_transients = pd.concat([df_transients, formation_channels_chunk[['channel']]], axis=1)

    return df_transients
    
BBH_mergers = BBH_pop.create_transient_population(BBH_creation_function, 'BBH')

<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Solution (click to reveal):</summary></b>

```python
# Additions to df_transients for our BBH merger transient population:
    df_transients['S1_state']  = history_chunk[mask]['S1_state']
    df_transients['S2_state']  = history_chunk[mask]['S2_state']
    df_transients['S1_mass'] = history_chunk[mask]['S1_mass']
    df_transients['S2_mass'] = history_chunk[mask]['S2_mass']
    df_transients['S1_spin'] = history_chunk[mask]['S1_spin']
    df_transients['S2_spin'] = history_chunk[mask]['S2_spin']
    # we distinguish the tilt of the spin to the orbit after the first and second SN.
    df_transients['S1_spin_orbit_tilt_merger'] = oneline_chunk['S1_spin_orbit_tilt_second_SN']
    df_transients['S2_spin_orbit_tilt_merger'] = oneline_chunk['S2_spin_orbit_tilt_second_SN']
    df_transients['orbital_period'] = history_chunk[mask]['orbital_period']
    df_transients['eccentricity'] = history_chunk[mask]['eccentricity']

    df_transients['chirp_mass'] = m_chirp(history_chunk[mask]['S1_mass'], history_chunk[mask]['S2_mass'])
    df_transients['mass_ratio'] = mass_ratio(history_chunk[mask]['S1_mass'], history_chunk[mask]['S2_mass'])
    df_transients['chi_eff'] = chi_eff(history_chunk[mask]['S1_mass'],
                                       history_chunk[mask]['S2_mass'],
                                       history_chunk[mask]['S1_spin'],
                                       history_chunk[mask]['S2_spin'],
                                       oneline_chunk['S1_spin_orbit_tilt_second_SN'],
                                       oneline_chunk['S2_spin_orbit_tilt_second_SN'])

plt.tight_layout()
```
    
</details>

`TransientPopulation` is a child of the `Population` class and now contains the extra information about the transient defined by the creation function above. Running `BBH_mergers.population` accesses the new `df_transients` dataframe that you specified above. 

In [ ]:
BBH_mergers.population

### 2.2 - Calculating model weights

You have selected merging binary black holes from a pre-computed run with a specific initial conditions, such as an initial orbial period distribution, initial mass function, etc.

Now that we have our transient population, let's calculate values that reweight each system in our population based on variations to the initial conditions. 

We'll first calculate the weights of each system (in units $M_\odot^{-1}$) using the sampled mass, and then when considering the full IMF. The `calculate_model_weights` method takes in a nametag for the weighting scheme as well as a dictionary specifying how we'd like to reweight our population. These weights can be accessed from the `TransientPopulation` class by the method `.model_weights`. 

In [ ]:
# these are the current initial conditions (per metallicity) in your population

print(BBH_mergers.ini_params)
_ = BBH_mergers.calculate_model_weights('sampled', population_parameters=BBH_mergers.ini_params)
BBH_mergers.model_weights()

You'll see that the model weights for all of the systems are the same. This makes sense, since we're reweighting to the same initial conditions that we sampled from! 

Let's now try calculating the model weights when we consider a population that accounts for the full IMF by changing `'primary_mass_min': 0.01` and `'primary_mass_min': 200`. We'll also specify a `q_min` and `q_max`, make the minimum secondary mass `0.01*0.05`, set the binary fraction to 0.7, and bring the `orbital_period_min` down to 0.5. 

In [ ]:
full_IMF = {'number_of_binaries': 10000,
 'binary_fraction_scheme': 'const',
 'binary_fraction_const': 0.7,
 'star_formation': 'burst',
 'max_simulation_time': 13800000000.0,
 'primary_mass_scheme': 'Kroupa2001',
 'primary_mass_min': 0.01,
 'primary_mass_max': 200.0,
 'secondary_mass_scheme': 'flat_mass_ratio',
 'secondary_mass_min': 0.01*0.05,
 'secondary_mass_max': 200.0,
 'q_min':0,
 'q_max':1,
 'orbital_scheme': 'period',
 'orbital_period_scheme': 'Sana+12_period_extended',
 'orbital_period_min': 0.5,
 'orbital_period_max': 6000.0,
 'eccentricity_scheme': 'zero'}

_ = BBH_mergers.calculate_model_weights('full_IMF', population_parameters=full_IMF)
BBH_mergers.model_weights()

We see some differences in the model weights for each system, but the big change is that the weights are quite a bit lower (by a factor of about 7.5)! This is because we're now account for the unsampled parts of the IMF, which contributes about 5x more stellar mass than the part of the IMF that we sampled (>7 $M_\odot$). We apply many variations to our initial conditions, such as the IMF, mass pairing, orbital period distribution, and binary fraction. These will lead to differences in the model weights you calculated. As an illustrative example, let's take a look at a number of different variations to the initial mass function. 

In [ ]:
# adapted from code at https://commons.wikimedia.org/wiki/File:Plot_of_various_initial_mass_functions.svg

import matplotlib.pyplot as plt
import numpy
from numpy import exp, log10 as log

def salpeter55(m):
	alpha = 2.35
	return m**-alpha

def millerscalo79(m):
	return numpy.where(m > 1, salpeter55(m), salpeter55(1))

def chabrier03individual(m):
	k = 0.158 * exp(-(-log(0.08))**2/(2 * 0.69**2))
	return numpy.where(m <= 1,\
	        0.158*(1./m) * exp(-(log(m)-log(0.08))**2/(2 * 0.69**2)),\
	        k*m**-2.3)

def chabrier03system(m):
	k = 0.086 * exp(-(-log(0.22))**2/(2 * 0.57**2))
	return numpy.where(m <= 1,\
	        0.086*(1./m) * exp(-(log(m)-log(0.22))**2/(2 * 0.57**2)),\
	        k*m**-2.3)

def kroupa01(m):
	return numpy.where(m<0.08, m**-0.3, numpy.where(m < 0.5, 0.08**-0.3 * (m/0.08)**-1.3, 0.08**-0.3 * (0.5/0.08)**-1.3 * (m/0.5)**-2.3))

def kroupa01_topheavy(m):
	return numpy.where(m<0.08, m**-0.3, numpy.where(m < 0.5, 0.08**-0.3 * (m/0.08)**-1.3, 0.08**-0.3 * (0.5/0.08)**-1.3 * (m/0.5)**-1.3))

def kroupa01_botheavy(m):
	return numpy.where(m<0.08, m**-1.3, numpy.where(m < 0.5, 0.08**-1.3 * (m/0.08)**-2.3, 0.08**-0.3 * (0.5/0.08)**-2.3 * (m/0.5)**-3.3))

plt.figure(figsize=(4,4))
m = numpy.logspace(np.log10(0.08), 2, 400)

for label, imf in zip('Salpeter55 MillerScalo79 Chabrier03individual Chabrier03system Kroupa01 Kroupa01_topheavy Kroupa01_bottomheavy'.split(),\
        [salpeter55, millerscalo79, kroupa01, chabrier03individual, chabrier03system, kroupa01_topheavy, kroupa01_botheavy]):
	plt.plot(m, imf(m)/imf(1), label=label)

plt.gca().set_yscale('log')
plt.gca().set_xscale('log')
plt.xlim(0.08, 100)
plt.ylim(1e-6, 1e4)
plt.legend(loc='best', prop=dict(size=8))
plt.xlabel('Mass [Solar mass]')
plt.ylabel(r'Mass Function $\xi(m)\Delta m$')

Let's try one more resampling where we change some more things with the IMF. We can actually specify the powerlaw indices for the broken power law functional form of the Kroupa mass function by adding another key to the dictionary: `'Kroupa2001' = {'alpha1': 0.3, 'alpha2': 1.3, 'alpha3': 1.3}`. Let's try it out below!

In [ ]:
topheavy_IMF = {'number_of_binaries': 10000,
 'binary_fraction_scheme': 'const',
 'binary_fraction_const': 0.7,
 'star_formation': 'burst',
 'max_simulation_time': 13800000000.0,
 'primary_mass_scheme': 'Kroupa2001',
 'Kroupa2001': {'alpha1': 0.3, 'alpha2': 1.3, 'alpha3': 1.3}, 
 'primary_mass_min': 0.01,
 'primary_mass_max': 200.0,
 'secondary_mass_scheme': 'flat_mass_ratio',
 'secondary_mass_min': 0.01*0.05,
 'secondary_mass_max': 200.0,
 'q_min':0,
 'q_max':1,
 'orbital_scheme': 'period',
 'orbital_period_scheme': 'Sana+12_period_extended',
 'orbital_period_min': 0.5,
 'orbital_period_max': 6000.0,
 'eccentricity_scheme': 'zero'}

_ = BBH_mergers.calculate_model_weights('topheavy_IMF', population_parameters=topheavy_IMF)
BBH_mergers.model_weights()

Let's also compare the cumulative efficiency (over all metallicities) between two of our initial conditions assumptions. 

In [ ]:
for samp in ['full_IMF', 'topheavy_IMF']:
    print('{:s}: {:0.2e} Msun^-1'.format(samp, np.sum(BBH_mergers.efficiency(samp).values)))

We wrote a similar plotting script out before, but there's also a handy way to plot the merger efficiency as a function of metallicity using POSYDON plotting functions. We can even separate it based on individual channels of BBH formation! Note that the plot will look a little ratty due to the relatively small number of sources that we modeled. 

In [ ]:
BBH_mergers.plot_efficiency_over_metallicity('full_IMF', channels=True)

In the next lab, we will use the `TransientPopulation` class and model weights that we learned here to generated predicted rate densities of transient events, explore variations in star formation histories, and apply selection effects to get detectable rates. 
___

## 3: Exercise

<div class="alert alert-success">

## EXERCISE

Luminous Red Novae are transients often associated with stellar mergers and mass ejection in common envelope. 

Use what you learned in this lab to create a new transient population: __potential Luminous Red Novae (LRN) progenitors__

1. Create a new `TransientPopulation` that targets these systems: a binary system that went through a common envelope event, regardless of whether the event ended in a failed common envelope merger or not.
2. Only select common envelope events where the two objects are stars, i.e. not a compact object.   
3. Collect information about the evolutionary state of the donor, the state of the inspiralling object, their metallicity, and radius.
4. Add weights to these systems using the `calculate_model_weights` method for the `TransientPopulation` class, and the `full_IMF` dictionary above
5. Plot the (weighted) formation efficiency (in units Msun^-1) for these systems.
6. Plot a distribution of S1_states for this population.


**Bonus**: If we are selecting progenitors solely based off a common envelope phase, it may be the case that one system produces potential LRN progenitors more than once: common envelope with two stars, common envelope with a compact object and star, common envelope with a compact object and post-common envelope star. This is an active field of study! 

The transient function assumes that the event of interest is a *transient* (happens only once). 

1. Create a function that carefully selects each unique LRN progenitor.
2. Repeat 3-6 above.
   
</div>

<div class="alert alert-warning" style="margin-top: 20px">
<details>

<b><summary>Solution (click to reveal):</summary></b>

```python
import numpy as np
import pandas as pd
from posydon.popsyn.synthetic_population import Population



# read in population files
print("Reading in population files and selection star-CE events...")
pop_files = ['1e-04_Zsun_population.h5',
             '1e-03_Zsun_population.h5',
             '1e-02_Zsun_population.h5',
             '1e-01_Zsun_population.h5',
             '2e-01_Zsun_population.h5',
             '4.5e-01_Zsun_population.h5',
             '1e+00_Zsun_population.h5',
             '2e+00_Zsun_population.h5']

def CE_with_two_stars_indices(pop):
    """
    Function to find the indices of common envelope systems when the inspiralling object is a star
    
    Assumes that RLO2->CE does not occur with a star as the inspiralling object.
    This may happen if there is reverse mass transfer, where RLO1 occurs onto S2, S2 then expands, 
    and RLO2 occurs onto S1 with S1 still being a star. 
    
    Be careful of edge cases! Alternately, you can select by common envelope and S1_state and S2_states. 
    
    Returns list of indices in the population `pop` where the binary system led to a common envelope with a star
    """
    # WRITE YOUR CODE HERE TO EXTRACT INDICES OF COMMON ENVELOPE EVENTS WITH A STAR
    selected_indices = pop.history.select(where='state == RLO1 & event == oCE1').index
    return list(selected_indices)


for file in pop_files:
    pop = Population(file)

    # function that gets the indices of systems that merged during CE with a star
    selected_indices_star = CE_with_two_stars_indices(pop)
    print(f'File: {file}, Number of systems that went through common envelope with two stars: {len(selected_indices_star)}')

    # let's also determine the formation channels for each system so we have them for later
    pop.calculate_formation_channels(mt_history=True)

    # export these systems to a new file called 'BBH_mergers.h5'
    pop.export_selection(selected_indices_star, 'CE_star.h5', append=True)



# create transient population
print("\n\n\nCreating transient population...")
CE_star_pop = Population('CE_star.h5')

def CE_star_creation_function(history_chunk, oneline_chunk, formation_channels_chunk):
    '''A CE creation function to create a transient population of CE events.'''

    indices = oneline_chunk.index.to_numpy() 
    df_transients = pd.DataFrame(index = indices)

    mask = (history_chunk['state'] == 'RLO1') & (history_chunk['event'] == 'oCE1') 
    df_transients['time'] = history_chunk[mask]['time']
    df_transients['metallicity'] = oneline_chunk['metallicity']

    df_transients['S1_mass'] = history_chunk[mask]['S1_mass']
    df_transients['S2_mass'] = history_chunk[mask]['S2_mass']
    
    df_transients['S1_state'] = history_chunk[mask]['S1_state']
    df_transients['S2_state'] = history_chunk[mask]['S2_state']
    
    df_transients['S1_log_R'] = history_chunk[mask]['S1_log_R']
    df_transients['S2_log_R'] = history_chunk[mask]['S2_log_R']

    # include formation channel
    df_transients = pd.concat([df_transients, formation_channels_chunk[['channel']]], axis=1)
    
    return df_transients
CE_star = CE_star_pop.create_transient_population(CE_star_creation_function, 'CE_star')



# calculate model weights
print("\n\n\nCalculating model weights...")

_ = CE_star.calculate_model_weights('full_IMF', population_parameters=full_IMF)
CE_star.model_weights()



# plot transient formation efficiency
print("\n\n\nPlotting efficiency...")
CE_star.plot_efficiency_over_metallicity('full_IMF', channels=True)



# plot distribution of S1 state at the onset of CE
print("\n\n\nPlotting distribution of S1 state...")
states = CE_star.population['S1_state']
counts = pd.Series(states).value_counts()

# Plot as a bar chart
plt.bar(counts.index, counts.values)
plt.xticks(rotation=80)
plt.xlabel("S1_state")
plt.ylabel("Count")
plt.title("Distribution of S1_state")
plt.show()
```
    
</details>